#### Notes
1. Get the max_length as small as possible.
2. Do not pass windows as input. This carries a lot of useless information.

3. Write input and labels to different files for each batch. Pickle the numpy arrays. 
4. Make window size 0 and see what it writes to inputs_batch file. # Window size = 0 still produces 6 integers. This means windowing is happening even without considering the window size.  On the other hand, padding attaches a list of two zeros when window size = 0. This implies that Padding has no problems, but either fix windowing. Extraacting center word from each 6-word llist is not going to get START  and END tokens.

An epoch is a complete run through the entire data. Within an epoch, there can be multiple training iterations. Each of those iterations take batch_size data and train on it. Num iterations per epoch = data size / batch size.

The next step is to find out how batches are formed from the central data. Where is the central data.

5. Write labels also to a file.
5. See how many batch files get created. 
6. Find a way to evaluate them.

#### load_and_preprocess_data method description
The follosing acrivities happen just once per run.
1. Master data is a list of tuples. Each tuple contains a sentence tokens list and a classes list.
2. Then a dictionary with {word:id} format is built.
3. The master data is then turned into it's numeric counter part. It still has a list of tuples. The first list in a tuple still contains words, but in numeric form and with it's part-of-speech info included.

#### NER_Model.fit method description-> 

1. fit method takes raw data as input. Raw data is simply  a list of tuples. The first list in a tuple still contains words, but in numeric form and with it's part-of-speech info included.
2. preprocess_sequence_data is called just once per run. An inner function "featurize_windows" creates the windows . 
    a) This creates windows of input alone, not the labels. How does it work with window size=0? "featurize_windows" function still creates 1-sized window even with window-size=0!!
    b) window_size=2: 1-windowed data. No matter what window_size is set to, it always creates 1-sized input. This method call can be avoided.


In [1]:
import pickle
import tensorflow as tf
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open("F:/Jupyter/NLP/assignment3/train_examples_raw.pkl","rb") as f:
    train_examples_raw = pickle.load(f)
    
with open("F:/Jupyter/NLP/assignment3/dev_set_raw.pkl","rb") as f:
    dev_set_raw = pickle.load(f)

In [3]:
def pad_sequences(data, max_length):
    """Ensures each input-output seqeunce pair in @data is of length
    @max_length by padding it with zeros and truncating the rest of the
    sequence.

    TODO: In the code below, for every sentence, labels pair in @data,
    (a) create a new sentence which appends zero feature vectors until
    the sentence is of length @max_length. If the sentence is longer
    than @max_length, simply truncate the sentence to be @max_length
    long.
    (b) create a new label sequence similarly.
    (c) create a _masking_ sequence that has a True wherever there was a
    token in the original sequence, and a False for every padded input.

    Example: for the (sentence, labels) pair: [[4,1], [6,0], [7,0]], [1,
    0, 0], and max_length = 5, we would construct
        - a new sentence: [[4,1], [6,0], [7,0], [0,0], [0,0]]
        - a new label seqeunce: [1, 0, 0, 4, 4], and
        - a masking seqeunce: [True, True, True, False, False].

    Args:
        data: is a list of (sentence, labels) tuples. @sentence is a list
            containing the words in the sentence and @label is a list of
            output labels. Each word is itself a list of
            @n_features features. For example, the sentence "Chris
            Manning is amazing" and labels "PER PER O O" would become
            ([[1,9], [2,9], [3,8], [4,8]], [1, 1, 4, 4]). Here "Chris"
            the word has been featurized as "[1, 9]", and "[1, 1, 4, 4]"
            is the list of labels. 
        max_length: the desired length for all input/output sequences.
    Returns:
        a new list of data points of the structure (sentence', labels', mask).
        Each of sentence', labels' and mask are of length @max_length.
        See the example above for more details.
    """
    ret = []

    # Use this zero vector when padding sequences.
    #Shailaja: TEST ONLY! REMOVE LATER.
    #zero_vector = [0] * Config.n_features
    zero_vector = [0] * 2 #Config.n_word_features
    zero_label = 4 # corresponds to the 'O' tag

    for sentence, labels in data:
        ### YOUR CODE HERE (~4-6 lines)
        new_sentence = list(sentence)
        new_labels = list(labels)

        # Create a sentence mask
        sentence_mask = [True] * len(sentence)
        false_value = False

        # Truncate longer sentences and labels.
        if len(sentence) > max_length: 
            new_sentence = new_sentence[0:max_length]
            new_labels = new_labels[0:max_length]
            sentence_mask = sentence_mask[0:max_length]
        else:
            padding_length = max_length - len(sentence)
            input_padding = [zero_vector] * padding_length
            output_padding = [zero_label] * padding_length
            mask_padding = [false_value] * padding_length

            new_sentence.extend(input_padding)
            new_labels.extend(output_padding)
            sentence_mask.extend(mask_padding)

        ret.append((new_sentence,new_labels,sentence_mask))
        ### END YOUR CODE ###
    return ret

In [4]:
ret = pad_sequences(data=train_examples_raw, max_length=32)

In [5]:
ret

[([[924, 10001],
   [10006, 10000],
   [186, 10002],
   [578, 10000],
   [7, 10000],
   [3800, 10000],
   [196, 10002],
   [5580, 10000],
   [3, 10003],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0]],
  [1,
   4,
   3,
   4,
   4,
   4,
   3,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4],
  [True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False]),
 ([[717, 10002],
   [1771, 10002],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0

In [6]:
# concatenate the words, which are now lists, of a sentence into a single list.
len(ret)
joinedSentencesRet = list()
for sentence, labels, masks in ret:
    newSentence = list()
    #[newSentence.extend(word) for word in sentence]
    newSentence = np.array([word for word in sentence])
    joinedSentencesRet.append((newSentence,labels,masks))

In [7]:
#len(joinedSentencesRet)
joinedSentencesRet[0][0].shape

(32, 2)

In [8]:
## Check how many sentences are losing info due to max_length =32 restricion.
len(ret) #14041 sentences
counter = 0
for sentence, label, mask in ret:
    try:
        if mask.index(False):
            counter +=1
    except:
        pass
counter # 12464 sentences have the zero padding.

12464

In [9]:
del ret

In [10]:
with open("F:/Jupyter/NLP/assignment3/embeddings.pkl","rb") as f:
    embeddings = pickle.load(f)
embeddings.shape

(10008, 50)

In [11]:
n_word_features = 2
batch_size = 512
hidden_size = 300
max_length = 32
embedding_size = 50
n_classes = 5
n_epochs = 50
dropout_rate = 0.5

In [12]:
def minibatches(data, batch_size, shuffle=True):
    batches = [np.array(col) for col in zip(*data)]
    return get_minibatches(batches, batch_size, shuffle)

def get_minibatches(data, minibatch_size, shuffle=True):
    list_data = type(data) is list and (type(data[0]) is list or type(data[0]) is np.ndarray)
    data_size = len(data[0]) if list_data else len(data)
    indices = np.arange(data_size)
    if shuffle:
        np.random.shuffle(indices)
    for minibatch_start in np.arange(0, data_size, minibatch_size):
        minibatch_indices = indices[minibatch_start:minibatch_start + minibatch_size]
        yield [minibatch(d, minibatch_indices) for d in data] if list_data \
            else minibatch(data, minibatch_indices)
            
def minibatch(data, minibatch_idx):
    return data[minibatch_idx] if type(data) is np.ndarray else [data[i] for i in minibatch_idx]

In [13]:
'''
def get_rnn_layer_output(rnn_layer, embeddings):
    output, state = rnn_layer(embeddings)
    return (output, state)

def get_output_layer_output(rnn_output,W,b2, dropout_placeholder):
    dropout_layer = tf.nn.dropout(rnn_output, keep_prob=1-dropout_placeholder, name="dropout_layer")
    return tf.matmul(dropout_layer,W) + b_2

def compute_loss(labels_placeholder, output_layer_output):
    return tf.nn.sparse_softmax_cross_entropy_with_logits(labels,output_layer_output, name="logloss")
'''
def get_batch_embeddings(input_placeholder,embeddings):
    embedding_len = embeddings.shape[1]
    batch_embeddings= tf.nn.embedding_lookup(params=embeddings, ids=input_placeholder)
    return tf.reshape(batch_embeddings,shape=(-1,max_length, n_word_features * embedding_len))

In [14]:
# Testing get_embeddings block
'''
dummyplaceholder = tf.placeholder(dtype=tf.int32,shape=(None,max_length, n_word_features), name="placeholder")
batch_embeddings = get_batch_embeddings(input_placeholder=dummyplaceholder,embeddings=embeddings)

tf.reshape(batch_embeddings,shape=(-1,max_length, n_word_features * embedding_size))
'''

'\ndummyplaceholder = tf.placeholder(dtype=tf.int32,shape=(None,max_length, n_word_features), name="placeholder")\nbatch_embeddings = get_batch_embeddings(input_placeholder=dummyplaceholder,embeddings=embeddings)\n\ntf.reshape(batch_embeddings,shape=(-1,max_length, n_word_features * embedding_size))\n'

In [15]:
# Test the sizes of mini-batches
'''
for mini_batch in minibatches(joinedSentencesRet, batch_size, shuffle=True):
    inputs_batch, labels_batch, mask_batch = mini_batch
    print(type(inputs_batch))
    print(inputs_batch.shape)
    break
'''

'\nfor mini_batch in minibatches(joinedSentencesRet, batch_size, shuffle=True):\n    inputs_batch, labels_batch, mask_batch = mini_batch\n    print(type(inputs_batch))\n    print(inputs_batch.shape)\n    break\n'

In [16]:
def get_output_layer_output(dropout_layer,W,b2,preds):
    for time_step in range(max_length):
        batch_of_rnn_output = tf.slice(dropout_layer,begin=[0,time_step,0],size=[-1,1,-1])
        reshaped_batch_of_rnn_output = tf.reshape(batch_of_rnn_output,shape=(-1,hidden_size))
        output_layer_output = tf.matmul(reshaped_batch_of_rnn_output,W) + b2
        preds.append(output_layer_output)
                
    new_preds = tf.stack(preds)
    final_preds = tf.reshape(new_preds,shape=(-1,max_length,n_classes))
    return final_preds

In [17]:
def apply_mask(to_mask,mask_placeholder):
    print("masking")
    return tf.boolean_mask(to_mask,mask_placeholder,name="masking_predictions")#,axis=1

In [18]:

    # If I start running epochs by passing feed_dict, that would mean recomputing the loss function and minimizing it. They have to functions computed before entering the loop.

#rnn_layer()

#tf.contrib.layers.xavier_initializer()
#rnn_output  
#state


In [21]:

# Placeholders
input_placeholder = tf.placeholder(dtype=tf.int32,shape=(None,max_length, n_word_features), name="input_placeholder")
labels_placeolder = tf.placeholder(dtype=tf.int32,shape=(None, max_length), name="labels_placeholder")
mask_placeholder = tf.placeholder(dtype=tf.bool, shape=(None, max_length), name="mask_placeholder")
dropout_placeholder = tf.placeholder(dtype=tf.float32, shape=None, name="dropout_placeholder")

preds = []

# Declare graph variables
with tf.variable_scope("rnn_scope", reuse=tf.AUTO_REUSE) as sc:
    W = tf.get_variable(name="W",shape=(hidden_size,n_classes),dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable(name="b2",shape=(n_classes,),dtype=tf.float32,initializer=tf.initializers.zeros())
    
    batch_embeddings = get_batch_embeddings(input_placeholder,embeddings)
    state_batch_size = tf.shape(batch_embeddings)[0]
    state = tf.zeros(shape=(state_batch_size,hidden_size),dtype=tf.float32, name="state_vector")
    # This is one of the quirks of  tf.keras.layers.SimpleRNNCell
    states = [state]

    # Build a graph:
    rnn_layer = tf.keras.layers.SimpleRNNCell(hidden_size,activation="tanh",                                  kernel_initializer=tf.contrib.layers.xavier_initializer(),recurrent_initializer=tf.contrib.layers.xavier_initializer(),dropout=0)
    #rnn_layer = tf.keras.layers.LSTMCell(hidden_size)
    
    for time_step in range(max_length):
        batch_of_words = tf.slice(batch_embeddings,begin=[0,time_step,0],size=[-1,1,-1])
        reshaped_batch_of_words = tf.reshape(batch_of_words,shape=(-1,n_word_features * embedding_size))
        #rnn_output, states = rnn_layer(reshaped_batch_of_words,states)
        rnn_output, states = rnn_layer(reshaped_batch_of_words)
        
        o_t_dropout = tf.nn.dropout(rnn_output,keep_prob=1-dropout_rate,name="h_dropout")
        o_t_logits = tf.matmul(o_t_dropout, W) + b2 
        preds.append(o_t_logits)
        
    new_preds = tf.stack(preds)
    final_preds = tf.reshape(new_preds,shape=(-1,max_length,n_classes))

    masked_preds = apply_mask(final_preds,mask_placeholder)
    masked_labels = apply_mask(labels_placeolder,mask_placeholder)

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=masked_labels,logits=masked_preds, name="logloss")) 
    minimize_loss = tf.train.AdamOptimizer(0.01).minimize(loss)
    
    init = tf.global_variables_initializer()
    with tf.Session() as s:
        s.run(init)
        for epoch in range(n_epochs):
            print("Epoch-",epoch,"=======================================")
            for mini_batch in minibatches(joinedSentencesRet, batch_size, shuffle=True):
                inputs_batch, labels_batch, mask_batch = mini_batch
                #input_batch_embeddings = get_batch_embeddings(inputs_batch,embeddings)
                feed_dict = dict()
                feed_dict[input_placeholder] = inputs_batch
                feed_dict[labels_placeolder] = labels_batch
                feed_dict[mask_placeholder] = mask_batch
                feed_dict[dropout_placeholder] = dropout_rate
                
                loss_val,_ = s.run([loss,minimize_loss],feed_dict)
            print("loss_val",loss_val)


TypeError: call() missing 1 required positional argument: 'states'

# Placeholders
input_placeholder = tf.placeholder(dtype=tf.int32,shape=(None,max_length, n_word_features), name="input_placeholder")
labels_placeolder = tf.placeholder(dtype=tf.int32,shape=(None, max_length), name="labels_placeholder")
mask_placeholder = tf.placeholder(dtype=tf.bool, shape=(None, max_length), name="mask_placeholder")
dropout_placeholder = tf.placeholder(dtype=tf.float32, shape=None, name="dropout_placeholder")

preds = []

# Declare graph variables
with tf.variable_scope("rnn_scope", reuse=tf.AUTO_REUSE) as sc:
    W = tf.get_variable(name="W",shape=(hidden_size,n_classes),dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable(name="b2",shape=(n_classes,),dtype=tf.float32,initializer=tf.initializers.zeros())
    
    batch_embeddings = get_batch_embeddings(input_placeholder,embeddings)
    state_batch_size = tf.shape(batch_embeddings)[0]
    state = tf.zeros(shape=(state_batch_size,hidden_size),dtype=tf.float32, name="state_vector")
    # This is one of the quirks of  tf.keras.layers.SimpleRNNCell
    states = [state]

    # Build a graph:
    rnn_layer = tf.keras.layers.SimpleRNNCell(hidden_size,activation="sigmoid",                                  kernel_initializer=tf.contrib.layers.xavier_initializer(),recurrent_initializer=tf.contrib.layers.xavier_initializer(),dropout=0)
    rnn_output, states = rnn_layer(batch_embeddings,states)
    dropout_layer = tf.nn.dropout(rnn_output,keep_prob=1-dropout_placeholder,name="dropout_layer")
    # This for loop is to multiply the rnn output at each time step with W
    '''
    for time_step in range(max_length):
        batch_of_rnn_output = tf.slice(dropout_layer,begin=[0,time_step,0],size=[-1,1,-1])
        reshaped_batch_of_rnn_output = tf.reshape(batch_of_rnn_output,shape=(-1,hidden_size))
        output_layer_output = tf.matmul(reshaped_batch_of_rnn_output,W) + b2
        preds.append(output_layer_output)
                
    new_preds = tf.stack(preds)
    final_preds = tf.reshape(new_preds,shape=(-1,max_length,n_classes))
    '''
    #final_preds = get_output_layer_output(dropout_layer,W,b2,preds)
    final_preds = tf.matmul(dropout_layer,W) + b2 #tf.matmul(dropout_layer,W) + b2
    

    '''
    ##masking
    print("masking preds")
    masked_preds = tf.boolean_mask(final_preds,mask_placeholder,name="masking_predictions")#,axis=1
    print("masking labels")
    masked_labels = tf.boolean_mask(labels_placeolder,mask_placeholder,name="masking_labels")#,
    '''
    
    masked_preds = apply_mask(final_preds,mask_placeholder)
    masked_labels = apply_mask(labels_placeolder,mask_placeholder)

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=masked_labels,logits=masked_preds, name="logloss")) 
    minimize_loss = tf.train.AdamOptimizer().minimize(loss)
    
    init = tf.global_variables_initializer()
    with tf.Session() as s:
        s.run(init)
        for epoch in range(n_epochs):
            print("Epoch-",epoch,"=======================================")
            for mini_batch in minibatches(joinedSentencesRet, batch_size, shuffle=True):
                inputs_batch, labels_batch, mask_batch = mini_batch
                #input_batch_embeddings = get_batch_embeddings(inputs_batch,embeddings)
                feed_dict = dict()
                feed_dict[input_placeholder] = inputs_batch
                feed_dict[labels_placeolder] = labels_batch
                feed_dict[mask_placeholder] = mask_batch
                feed_dict[dropout_placeholder] = dropout_rate
                
                loss_val,_ = s.run([loss,minimize_loss],feed_dict)


In [ ]:
#batches = [np.array(col) for col in zip(*joinedSentencesRet)]
i=0
for col in zip(*joinedSentencesRet):
#for col in zip(*ret):
    a = np.array(col)
    print(a)
    print(a.shape)
    i+=1
    print("*****************")
print(i)

In [ ]:
del batches

In [ ]:
import os

In [ ]:
print(os.path.join(os.getcwd(), "ml"))
currentPath = "F:/Jupyter/NLP/assignment3/"
print(os.path.join(currentPath, "ml"))

In [ ]:
#os.getcwd()
#os.getcwd()+"/"